In [1]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
path = "pdffiles"
loader = DirectoryLoader(path, glob="**/*.pdf", loader_cls=PyPDFLoader, silent_errors=True)
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100, length_function=len, is_separator_regex=False)
texts = text_splitter.create_documents([pages[i].page_content for i in range(len(pages))])

In [8]:
pages[6].dict()

{'page_content': '133MODULE - 5\nContemporary World\nNotes\nHISTORY THE COLD WAR  ERA AND ITS POLITICS\nfrom the USA. In North Korea a pro-Soviet Government was set up. Neither the\nSoviet Union nor the U.S.A. recognised the governments which were opposed tothem. In 1950, North Korea invaded South Korea. The United Nations, whose per-\nmanent Security Council was dominated by capitalist states, declared North Korea\nthe aggressor and set up a unified UN command to repel the North Korean attack.General MacArthur of the USA was named its commander. The UN troops pushed\nNorth Korean forces out of South Korea and entered deep into the North Korean\nterritory, reaching the Chinese border. China then joined the North Korean troops topush the UN troops into South Korea. Ultimately an armistice was signed in 1953\nbringing to an end the threat of an open war. The Korean crisis was the first military\nstruggle of the Cold War. The USA and USSR and PRC did not engage in muchdirect combat with o

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

# from langchain_together import Together
# from langchain_together.embeddings import TogetherEmbeddings
# together_key = ""

# generator_llm = Together(
#     model="NousResearch/Nous-Hermes-2-Mixtral-8x7B-SFT",
#     temperature=0.7,
#     max_tokens=4000,
#     top_k=1,
#     together_api_key=together_key
# )
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k",openai_api_key='')
critic_llm = ChatOpenAI(
    model="explodinggradients/Ragas-critic-llm-Qwen1.5-GPTQ",
    openai_api_key="",
    openai_api_base="https://api.runpod.ai/v2/1lh33m6cyf1cue/openai/v1",
    max_tokens=2048,
    temperature=0,
)
embeddings = OpenAIEmbeddings(openai_api_key='')
# embeddings = TogetherEmbeddings(model="togethercomputer/m2-bert-80M-8k-retrieval")

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.33,
    multi_context: 0.33,
    reasoning: 0.34
}

testset = generator.generate_with_langchain_docs(texts, 15, distributions)
df = testset.to_pandas()
df[['question', 'ground_truth']]

In [6]:
import requests

def create_completion(base_url, prompt, api_key):
    completion_endpoint = base_url + "/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    data = {
        "model":"explodinggradients/Ragas-critic-llm-Qwen1.5-GPTQ",
        "prompt": prompt,
        "max_tokens": 2000,
        "temperature": 0.7,
    }
    response = requests.post(completion_endpoint, json=data, headers=headers)
    return response.json()

# Example usage:
base_url = "https://api.runpod.ai/v2/1lh33m6cyf1cue/openai/v1"
api_key = ""
prompt = "Once upon a time"
completion = create_completion(base_url, prompt, api_key)
print(completion)


{'choices': [{'finish_reason': 'length', 'index': 0, 'logprobs': None, 'text': ' there was a young woman who had a difficult time in her life. It was a time when she was very worried about her health and her ability to provide a life for her child.  One day, while out shopping, she met a wise old woman who asked her why she was so worried. The woman explained her problems. The woman who then asked her what she could do to help her. The woman answered, "The only thing I do is to be kind and considerate to other people and to be true to myself. "  The woman said, "That is all?\'  "The woman answered, "I have one more thing to add. I also have another child to care for in a very serious condition. I have not only to be kind and considerate to other people, but also to be kind and considerate to them in the outside world.  The woman said, "That is the most important part of the job. I have a 2-year-old child. The other one is 1128133616226233232263333333333334333333333333333333333333333333